In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import PredefinedSplit
import matplotlib.pyplot as plt
import warnings as wr
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from pathlib import Path

wr.filterwarnings('ignore')

work_dir = Path.cwd().parent.parent / 'Work'
df_web = pd.read_excel(work_dir / "summary_tests.xlsx")
df_app = pd.read_csv(work_dir / "summary_test.tsv", sep ='\t')
